In [115]:
import pandas as pd
from pathlib import Path
import numpy as np
path = Path("./data")
df = pd.read_csv(path / "Ad_table (extra).csv")
target = "Price"
X = df.drop(columns=[target, "Annual_Tax"])
y = df[target]

In [116]:
print(X)
X.shape

            Maker  Genmodel  Genmodel_ID   Adv_ID  Adv_year  Adv_month  \
0         Bentley    Arnage         10_1  10_1$$1      2018          4   
1         Bentley    Arnage         10_1  10_1$$2      2018          6   
2         Bentley    Arnage         10_1  10_1$$3      2017         11   
3         Bentley    Arnage         10_1  10_1$$4      2018          4   
4         Bentley    Arnage         10_1  10_1$$5      2017         11   
...           ...       ...          ...      ...       ...        ...   
268250  Westfield     Sport         97_1  97_1$$1      2018          5   
268251  Westfield     Sport         97_1  97_1$$2      2018          5   
268252      Zenos       E10         99_1  99_1$$1      2018          3   
268253      Zenos       E10         99_1  99_1$$2      2018          3   
268254      Zenos       E10         99_1  99_1$$3      2018          5   

         Color  Reg_year     Bodytype Runned_Miles  ... Fuel_type  \
0       Silver    2000.0       Saloon     

(268255, 22)

In [117]:
X[["Height", "Width", "Length", "Wheelbase"]] = X[["Height", "Width", "Length", "Wheelbase"]].astype(float).replace(0., np.nan)

In [118]:
X = X.dropna()

In [119]:
X.shape

(193593, 22)

In [120]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193593 entries, 1 to 268249
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Maker         193593 non-null  object 
 1    Genmodel     193593 non-null  object 
 2    Genmodel_ID  193593 non-null  object 
 3   Adv_ID        193593 non-null  object 
 4   Adv_year      193593 non-null  int64  
 5   Adv_month     193593 non-null  int64  
 6   Color         193593 non-null  object 
 7   Reg_year      193593 non-null  float64
 8   Bodytype      193593 non-null  object 
 9   Runned_Miles  193593 non-null  object 
 10  Engin_size    193593 non-null  object 
 11  Gearbox       193593 non-null  object 
 12  Fuel_type     193593 non-null  object 
 13  Engine_power  193593 non-null  float64
 14  Wheelbase     193593 non-null  float64
 15  Height        193593 non-null  float64
 16  Width         193593 non-null  float64
 17  Length        193593 non-null  float64
 18  Average_m

In [121]:
X.columns

Index(['Maker', ' Genmodel', ' Genmodel_ID', 'Adv_ID', 'Adv_year', 'Adv_month',
       'Color', 'Reg_year', 'Bodytype', 'Runned_Miles', 'Engin_size',
       'Gearbox', 'Fuel_type', 'Engine_power', 'Wheelbase', 'Height', 'Width',
       'Length', 'Average_mpg', 'Top_speed', 'Seat_num', 'Door_num'],
      dtype='object')

In [122]:
X.columns = X.columns.str.strip()

In [123]:
X["Genmodel_ID"].value_counts()

Genmodel_ID
94_9     4930
29_13    3332
29_14    2696
95_15    2466
29_27    2306
         ... 
62_22       1
62_5        1
63_2        1
62_9        1
27_11       1
Name: count, Length: 646, dtype: int64

In [124]:
cols = [
    "Maker",
    "Genmodel",
    "Adv_ID"
]
X = X.drop(cols, axis=1)

In [125]:
X["Adv_month"] = X["Adv_month"].clip(1, 12)
X["Reg_year"] = X["Reg_year"].clip(1960, 2025)

In [126]:
X["Adv_date"] = X["Adv_year"].astype(str) + "-" + X["Adv_month"].astype(str)+ "-1"
X["Adv_date"] = pd.to_datetime(X["Adv_date"]).dt.year
X["car_old"] = X["Adv_date"] - X["Reg_year"]
X["car_old"]

1         16.0
2         15.0
5         15.0
6         16.0
11        16.0
          ... 
268245    10.0
268246    11.0
268247     9.0
268248    14.0
268249     9.0
Name: car_old, Length: 193593, dtype: float64

In [127]:
X["Runned_Miles"] = pd.to_numeric(X["Runned_Miles"], errors="coerce")

In [128]:
X["Runned_Miles"].isna().sum()
X = X.dropna()

In [129]:
def eng_size(val):
    val = str(val).lower()
    if "l" in val:
        return float(val.replace("l", ""))
    elif "cc" in val:
        return float(val.replace("cc", "") / 1000)
    else:
        return pd.to_numeric(val, errors="coerce")
X["Engin_size"] = X["Engin_size"].apply(eng_size)

In [130]:
X["Engin_size"] = X["Engin_size"].dropna()

In [131]:
X[["Average_mpg", "Top_speed"]]

,Average_mpg,Top_speed
1,13.7 mpg,179 mph
2,14.7 mpg,155 mph
5,13.7 mpg,179 mph
6,13.7 mpg,179 mph
11,13.7 mpg,179 mph
...,...,...
268245,48.7 mpg,127 mph
268246,40.4 mpg,137 mph
268247,40.4 mpg,137 mph
268248,31.0 mpg,134 mph


In [132]:
X["Average_mpg"] = (
    X["Average_mpg"]
      .astype(str)
      .str.lower()
      .str.replace("mpg", "", regex=False)
      .pipe(pd.to_numeric, errors="coerce")
)

X["Top_speed"] = (
    X["Top_speed"]
      .astype(str)
      .str.lower()
      .str.replace("mph", "", regex=False)
      .pipe(pd.to_numeric, errors="coerce")
)

In [133]:
X.describe()

,Adv_year,Adv_month,Reg_year,Runned_Miles,Engin_size,Engine_power,Wheelbase,Height,Width,Length,Average_mpg,Top_speed,Seat_num,Door_num,Adv_date,car_old
count,193386.000000,193386.000000,193386.000000,1.933860e+05,193386.000000,193386.000000,193386.000000,193386.00000,193386.000000,193386.000000,193386.000000,193386.000000,193386.000000,193386.000000,193386.000000,193386.000000
mean,2017.954914,5.796624,2012.169557,5.110257e+04,1.898131,150.912762,2641.698236,1534.56714,1893.705051,4347.705341,50.977080,122.286515,4.901730,4.372555,2017.954914,5.785357
std,0.226806,2.052121,4.330798,4.289455e+04,0.760253,86.197615,178.503833,126.36008,154.220812,413.716473,13.798868,18.462668,0.862477,1.010019,0.226806,4.336587
min,2012.000000,1.000000,2000.000000,2.000000e+00,0.600000,44.000000,1420.000000,1112.00000,1475.000000,2500.000000,9.400000,11.000000,1.000000,2.000000,2012.000000,-2.000000
25%,2018.000000,4.000000,2009.000000,1.650000e+04,1.400000,99.000000,2511.000000,1458.00000,1775.000000,4052.000000,41.500000,109.000000,5.000000,4.000000,2018.000000,2.000000
50%,2018.000000,6.000000,2013.000000,4.400000e+04,1.600000,126.000000,2640.000000,1491.00000,1885.000000,4368.000000,50.400000,118.000000,5.000000,5.000000,2018.000000,5.000000
75%,2018.000000,8.000000,2016.000000,7.890000e+04,2.000000,177.000000,2754.000000,1620.00000,2020.000000,4659.000000,60.100000,131.000000,5.000000,5.000000,2018.000000,9.000000
max,2018.000000,12.000000,2018.000000,6.363342e+06,6.800000,740.000000,5246.000000,2660.00000,2690.000000,6165.000000,200.000000,226.000000,9.000000,5.000000,2018.000000,18.000000


In [134]:
X["MilesPerYear"] = X["Runned_Miles"] / (X["car_old"] + 1e-6)
X["ts to power"] = X["Top_speed"] / (X["Engine_power"] + 1e-6)
X["efficiency"] = X["Engin_size"] / (X["Average_mpg"] + 1e-6)

cols = [
    "Reg_year",
    "Adv_year",
    "Adv_month",
]
X = X.drop(cols, axis=1)

In [135]:
X = pd.get_dummies(X, columns=["Color", "Bodytype", "Gearbox", "Fuel_type"], drop_first=True)

In [136]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193386 entries, 1 to 268249
Data columns (total 55 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   Genmodel_ID                                193386 non-null  object 
 1   Runned_Miles                               193386 non-null  float64
 2   Engin_size                                 193386 non-null  float64
 3   Engine_power                               193386 non-null  float64
 4   Wheelbase                                  193386 non-null  float64
 5   Height                                     193386 non-null  float64
 6   Width                                      193386 non-null  float64
 7   Length                                     193386 non-null  float64
 8   Average_mpg                                193386 non-null  float64
 9   Top_speed                                  193386 non-null  int64  
 10  Seat_num     

In [137]:
X.to_parquet("encoded.parquet")